In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Data Exploration - Market Making Environment\n",
    "\n",
    "Explore the market environment and understand the dynamics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../python')\n",
    "\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "from env.market_env import MarketMakerEnv\n",
    "\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Environment Initialization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create environment\n",
    "env = MarketMakerEnv()\n",
    "\n",
    "print(f\"Observation Space: {env.observation_space}\")\n",
    "print(f\"Action Space: {env.action_space}\")\n",
    "print(f\"\\nInitial Price: ${env.initial_price:,.2f}\")\n",
    "print(f\"Max Inventory: {env.max_inventory}\")\n",
    "print(f\"Episode Length: {env.episode_length}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Simulate Random Trading Episodes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Run random agent\n",
    "n_episodes = 10\n",
    "results = []\n",
    "\n",
    "for ep in range(n_episodes):\n",
    "    obs, info = env.reset()\n",
    "    done = False\n",
    "    \n",
    "    episode_data = {\n",
    "        'prices': [],\n",
    "        'inventories': [],\n",
    "        'pnls': [],\n",
    "        'spreads': []\n",
    "    }\n",
    "    \n",
    "    while not done:\n",
    "        action = env.action_space.sample()\n",
    "        obs, reward, terminated, truncated, info = env.step(action)\n",
    "        done = terminated or truncated\n",
    "        \n",
    "        episode_data['prices'].append(info['mid_price'])\n",
    "        episode_data['inventories'].append(info['inventory'])\n",
    "        episode_data['pnls'].append(info['total_pnl'])\n",
    "    \n",
    "    results.append(episode_data)\n",
    "\n",
    "print(f\"✓ Simulated {n_episodes} episodes\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Analyze Price Dynamics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot price paths\n",
    "fig, axes = plt.subplots(2, 1, figsize=(14, 8))\n",
    "\n",
    "# Price paths\n",
    "for i, ep_data in enumerate(results[:5]):\n",
    "    axes[0].plot(ep_data['prices'], alpha=0.6, label=f'Episode {i+1}')\n",
    "\n",
    "axes[0].set_title('Price Evolution (Random Agent)', fontsize=14, fontweight='bold')\n",
    "axes[0].set_xlabel('Step')\n",
    "axes[0].set_ylabel('Mid Price ($)')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Price returns distribution\n",
    "all_prices = []\n",
    "for ep_data in results:\n",
    "    all_prices.extend(ep_data['prices'])\n",
    "\n",
    "returns = np.diff(all_prices) / all_prices[:-1]\n",
    "axes[1].hist(returns, bins=50, alpha=0.7, edgecolor='black')\n",
    "axes[1].set_title('Returns Distribution', fontsize=14, fontweight='bold')\n",
    "axes[1].set_xlabel('Return')\n",
    "axes[1].set_ylabel('Frequency')\n",
    "axes[1].axvline(x=0, color='r', linestyle='--', alpha=0.5)\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\nPrice Statistics:\")\n",
    "print(f\"  Mean: ${np.mean(all_prices):,.2f}\")\n",
    "print(f\"  Std: ${np.std(all_prices):,.2f}\")\n",
    "print(f\"  Return Mean: {np.mean(returns):.6f}\")\n",
    "print(f\"  Return Std: {np.std(returns):.6f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Inventory Management Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot inventory over time\n",
    "fig, axes = plt.subplots(2, 1, figsize=(14, 8))\n",
    "\n",
    "for i, ep_data in enumerate(results[:5]):\n",
    "    axes[0].plot(ep_data['inventories'], alpha=0.6, label=f'Episode {i+1}')\n",
    "\n",
    "axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)\n",
    "axes[0].set_title('Inventory Over Time (Random Agent)', fontsize=14, fontweight='bold')\n",
    "axes[0].set_xlabel('Step')\n",
    "axes[0].set_ylabel('Inventory (BTC)')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Inventory distribution\n",
    "all_inventories = []\n",
    "for ep_data in results:\n",
    "    all_inventories.extend(ep_data['inventories'])\n",
    "\n",
    "axes[1].hist(all_inventories, bins=50, alpha=0.7, edgecolor='black')\n",
    "axes[1].set_title('Inventory Distribution', fontsize=14, fontweight='bold')\n",
    "axes[1].set_xlabel('Inventory')\n",
    "axes[1].set_ylabel('Frequency')\n",
    "axes[1].axvline(x=0, color='r', linestyle='--', alpha=0.5)\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\nInventory Statistics:\")\n",
    "print(f\"  Mean: {np.mean(all_inventories):.6f}\")\n",
    "print(f\"  Max: {np.max(np.abs(all_inventories)):.6f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. PnL Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot PnL trajectories\n",
    "fig, ax = plt.subplots(figsize=(14, 6))\n",
    "\n",
    "final_pnls = []\n",
    "for i, ep_data in enumerate(results):\n",
    "    ax.plot(ep_data['pnls'], alpha=0.6, label=f'Episode {i+1}')\n",
    "    final_pnls.append(ep_data['pnls'][-1])\n",
    "\n",
    "ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)\n",
    "ax.set_title('PnL Evolution (Random Agent)', fontsize=14, fontweight='bold')\n",
    "ax.set_xlabel('Step')\n",
    "ax.set_ylabel('PnL ($)')\n",
    "ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "ax.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\nRandom Agent Performance:\")\n",
    "print(f\"  Mean Final PnL: ${np.mean(final_pnls):.2f}\")\n",
    "print(f\"  Std Final PnL: ${np.std(final_pnls):.2f}\")\n",
    "print(f\"  Win Rate: {np.sum(np.array(final_pnls) > 0) / len(final_pnls):.2%}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. State Space Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Collect observations\n",
    "observations = []\n",
    "obs, _ = env.reset()\n",
    "\n",
    "for _ in range(1000):\n",
    "    action = env.action_space.sample()\n",
    "    obs, _, terminated, truncated, _ = env.step(action)\n",
    "    observations.append(obs)\n",
    "    \n",
    "    if terminated or truncated:\n",
    "        obs, _ = env.reset()\n",
    "\n",
    "obs_array = np.array(observations)\n",
    "\n",
    "# Feature names\n",
    "feature_names = ['Inventory', 'Price', 'Spread', 'Volatility', \n",
    "                'Time', 'Recent PnL', 'Order Imbalance', 'Microprice']\n",
    "\n",
    "# Plot distributions\n",
    "fig, axes = plt.subplots(2, 4, figsize=(16, 8))\n",
    "axes = axes.flatten()\n",
    "\n",
    "for i, (ax, name) in enumerate(zip(axes, feature_names)):\n",
    "    ax.hist(obs_array[:, i], bins=30, alpha=0.7, edgecolor='black')\n",
    "    ax.set_title(name, fontweight='bold')\n",
    "    ax.set_xlabel('Value')\n",
    "    ax.set_ylabel('Frequency')\n",
    "    ax.grid(True, alpha=0.3)\n",
    "\n",
    "plt.suptitle('Observation Space Distributions', fontsize=16, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "- Environment simulates realistic market dynamics\n",
    "- Random agent has near-zero expected PnL\n",
    "- Inventory management is crucial for reducing risk\n",
    "- Reward function balances PnL, inventory risk, and spreads"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}